# 0.デフォルト実行　インポートとか関数とか  アンロードも


In [10]:
#  000-000
##　初回　必須　　

import numpy as np
import pandas as pd
import pickle
import io
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

#jupyterlabを使う場合、この2行はいらない
from jupyterthemes import jtplot
jtplot.style(theme="monokai")


In [11]:
#  000-001
#　ｒａｃｅ＿ｉｎｆｏに関するコード　処理必須
ar_code_list = {\
        "10":"直",\
        "11":"左",\
        "12":"左",\
        "13":"左",\
        "14":"左",\
        "15":"左",\
        "16":"左",\
        "17":"右",\
        "18":"右",\
        "19":"右",\
        "20":"右",\
        "21":"右",\
        "22":"右",\
        "23":"左",\
        "24":"右",\
        "25":"左",\
        "26":"右",\
        "27":"左",\
        "28":"右",\
        "29":"直",\
        "51":"障",\
        "52":"障",\
        "53":"障",\
        "54":"障",\
        "55":"障",\
        "56":"障",\
        "57":"障",\
        "58":"障",\
        "59":"障"\
    } 

gr_code_list = {\
        "10":"芝",\
        "11":"芝",\
        "12":"芝",\
        "13":"芝",\
        "14":"芝",\
        "15":"芝",\
        "16":"芝",\
        "17":"芝",\
        "18":"芝",\
        "19":"芝",\
        "20":"芝",\
        "21":"芝",\
        "22":"芝",\
        "23":"ダ",\
        "24":"ダ",\
        "25":"ダ",\
        "26":"ダ",\
        "27":"サ",\
        "28":"サ",\
        "29":"ダ",\
        "51":"障",\
        "52":"障",\
        "53":"障",\
        "54":"障",\
        "55":"障",\
        "56":"障",\
        "57":"障",\
        "58":"障",\
        "59":"障"\
    } 

class_code_list = {\
        "005":"１勝クラス",\
        "010":"２勝クラス",\
        "016":"３勝クラス",\
        "701":"新馬",\
        "703":"未勝利",\
        "999":"オープン"\
    } 

grade_code_list = {\
        "A":"Ｇ１",\
        "B":"Ｇ２",\
        "C":"Ｇ３",\
        "D":"その他",\
        "E":"その他",\
        "F":"その他",\
        "G":"その他",\
        "H":"その他",\
        "L":"リステッド",\
        " ":"その他"\
    } 

juryo_shubetsu_code_list = {\
        "0":"未設定",\
        "1":"ハンデ",\
        "2":"別定",\
        "3":"馬齢",\
        "4":"定量"\
    } 

babajotai_code_list = {\
        "0":"未設定",\
        "1":"良",\
        "2":"稍重",\
        "3":"重",\
        "4":"不良"\
    } 

tenko_code_list = {\
        "0":"未設定",\
        "1":"晴",\
        "2":"曇",\
        "3":"雨",\
        "4":"小雨",\
        "5":"雪",\
        "6":"小雪"\
    } 

kyoso_shubetsu_code_list = {\
        "00":"未設定",\
        "11":"サラ２歳",\
        "12":"サラ３歳",\
        "13":"サラ３歳以上",\
        "14":"サラ４歳以上",\
        "18":"サラ障害３歳以上",\
        "19":"サラ障害４歳以上",\
        "21":"アラブ２歳",\
        "22":"アラブ３歳",\
        "23":"アラブ３歳以上",\
        "24":"アラブ４歳以上"\
    } 


In [12]:
#  000-002-01
###　初回必須　　0020-01
from tqdm.notebook import tqdm

class HorseResults:
    def __init__(self, horse_info_):
        self.horse_results = horse_info_[['race_nengappi','kakutei_chakujun','tansho_odds','kakutoku_honshokin','kohan_3f']]
        self.preprocessing()
        
    def preprocessing(self):
        df = self.horse_results.copy()
        df['date'] = pd.to_datetime(df["race_nengappi"],format='%Y%m%d')
        df.drop(["race_nengappi"],axis=1,inplace=True)
        self.horse_results = df

    def average(self, horse_id_list, date, n_samples='all'):
        target_df = self.horse_results.loc[horse_id_list]
#        target_df = target_df.set_index('ketto_toroku_bango') 

        #過去何走分取り出すか指定
        if n_samples == 'all':
            filterd_df = target_df[target_df['date'] < date]    
        elif n_samples > 0:
            filterd_df = target_df[target_df['date'] < date].sort_values\
                ('date',ascending=False).groupby(level=0).head(n_samples)
        else:
            raise Exception('n_samples must be >0')
        average = filterd_df.groupby(level=0)[['kakutei_chakujun','kakutoku_honshokin','kohan_3f']].mean()
        return average.rename(columns=\
                              {'kakutei_chakujun':'chakujun_{}R'.format(n_samples),\
                               'kakutoku_honshokin':'shokin_{}R'.format(n_samples),\
                               'kohan_3f':'kohan_3f_{}R'.format(n_samples)})
    
    def merge(self, results, date, n_samples='all'):
        df = results[results['date'] == date]
        df.reset_index(inplace=True)
        horse_id_list = df['ketto_toroku_bango']
        merged_df = df.merge(self.average(horse_id_list, date, n_samples),\
                             left_on='ketto_toroku_bango',right_index=True,how='left')
        return merged_df
    
    def merge_all(self, results, n_samples='all'):
        date_list = results['date'].unique()
        merged_df = pd.concat([self.merge(results, date, n_samples) for date in tqdm(date_list)])
        return merged_df

In [13]:
#  000-002-02
###　関数２　使わない？？？　改良の余地あり
from tqdm.notebook import tqdm

class HorseResults2:
    def __init__(self, horse_info_):
        self.horse_results = horse_info_[['race_nengappi','kakutei_chakujun','tansho_odds','kakutoku_honshokin','kohan_3f']]
        self.preprocessing2()
        
    def preprocessing2(self):
        df = self.horse_results.copy()
        df['date'] = pd.to_datetime(df["race_nengappi"],format='%Y%m%d')
        df.drop(["race_nengappi"],axis=1,inplace=True)
        self.horse_results = df

    def high(self, horse_id_list, date, n_samples='all'):
        target_df = self.horse_results.loc[horse_id_list]
#        target_df = target_df.set_index('ketto_toroku_bango') 

        #過去何走分取り出すか指定
        if n_samples == 'all':
            filterd_df = target_df[target_df['date'] < date]    
        elif n_samples > 0:
            filterd_df = target_df[target_df['date'] < date].sort_values\
                ('date',ascending=False).groupby(level=0).head(n_samples)
        else:
            raise Exception('n_samples must be >0')
        high = filterd_df.groupby(level=0)[['kakutei_chakujun','kohan_3f']].min()
        return high.rename(columns=\
                              {'kakutei_chakujun':'chakujun_min_{}R'.format(n_samples),\
                               'kohan_3f':'kohan_3f_min_{}R'.format(n_samples)})
    
    def merge2(self, results, date, n_samples='all'):
        df = results[results['date'] == date]
        df.reset_index(inplace=True)
        horse_id_list = df['ketto_toroku_bango']
        merged_df = df.merge(self.high(horse_id_list, date, n_samples),\
                             left_on='ketto_toroku_bango',right_index=True,how='left')
        return merged_df
    
    def merge_all2(self, results, n_samples='all'):
        date_list = results['date'].unique()
        merged_df = pd.concat([self.merge2(results, date, n_samples) for date in tqdm(date_list)])
        return merged_df

In [14]:
#　000-003　関数
#　データ分割（トレーニング、テストデータ）
def split_data(df, test_size):
    sorted_id_list = df.sort_values("date").index.unique()
    train_id_list = sorted_id_list[: round(len(sorted_id_list) * (1 - test_size))]
    test_id_list = sorted_id_list[round(len(sorted_id_list) * (1 - test_size)) :]
    train = df.loc[train_id_list]
    test = df.loc[test_id_list]
    return train, test

In [16]:
#000-010
###　初回　必須　　0010-10
### アンロード
race_info_2013 = pd.read_pickle('data2/race_info_2013.pickle')
race_info_2014 = pd.read_pickle('data2/race_info_2014.pickle')
race_info_2015 = pd.read_pickle('data2/race_info_2015.pickle')
race_info_2016 = pd.read_pickle('data2/race_info_2016.pickle')
race_info_2017 = pd.read_pickle('data2/race_info_2017.pickle')
race_info_2018 = pd.read_pickle('data2/race_info_2018.pickle')
race_info_2019 = pd.read_pickle('data2/race_info_2019.pickle')
race_info_2020 = pd.read_pickle('data2/race_info_2020.pickle')
race_info_2021 = pd.read_pickle('data2/race_info_2021.pickle')
race_info_2022 = pd.read_pickle('data2/race_info_2022.pickle')
race_info_2023 = pd.read_pickle('data2/race_info_2023.pickle')
race_info_2024 = pd.read_pickle('data2/race_info_2024.pickle')
race_info = pd.concat([
            race_info_2013,\
            race_info_2014,\
            race_info_2015,\
            race_info_2016,\
            race_info_2017,\
            race_info_2018,\
            race_info_2019,\
            race_info_2020,\
            race_info_2021,\
            race_info_2022,\
            race_info_2023,\
            race_info_2024\
            ])

race_results_2013 = pd.read_pickle('data2/race_results_2013.pickle')
race_results_2014 = pd.read_pickle('data2/race_results_2014.pickle')
race_results_2015 = pd.read_pickle('data2/race_results_2015.pickle')
race_results_2016 = pd.read_pickle('data2/race_results_2016.pickle')
race_results_2017 = pd.read_pickle('data2/race_results_2017.pickle')
race_results_2018 = pd.read_pickle('data2/race_results_2018.pickle')
race_results_2019 = pd.read_pickle('data2/race_results_2019.pickle')
race_results_2020 = pd.read_pickle('data2/race_results_2020.pickle')
race_results_2021 = pd.read_pickle('data2/race_results_2021.pickle')
race_results_2022 = pd.read_pickle('data2/race_results_2022.pickle')
race_results_2023 = pd.read_pickle('data2/race_results_2023.pickle')
race_results_2024 = pd.read_pickle('data2/race_results_2024.pickle')
race_results = pd.concat([
            race_results_2013,\
            race_results_2014,\
            race_results_2015,\
            race_results_2016,\
            race_results_2017,\
            race_results_2018,\
            race_results_2019,\
            race_results_2020,\
            race_results_2021,\
            race_results_2022,\
            race_results_2023,\
            race_results_2024\
            ])

horse_info_2006 = pd.read_pickle('data2/horse_info_2006.pickle')
horse_info_2007 = pd.read_pickle('data2/horse_info_2007.pickle')
horse_info_2008 = pd.read_pickle('data2/horse_info_2008.pickle')
horse_info_2009 = pd.read_pickle('data2/horse_info_2009.pickle')
horse_info_2010 = pd.read_pickle('data2/horse_info_2010.pickle')
horse_info_2011 = pd.read_pickle('data2/horse_info_2011.pickle')
horse_info_2012 = pd.read_pickle('data2/horse_info_2012.pickle')
horse_info_2013 = pd.read_pickle('data2/horse_info_2013.pickle')
horse_info_2014 = pd.read_pickle('data2/horse_info_2014.pickle')
horse_info_2015 = pd.read_pickle('data2/horse_info_2015.pickle')
horse_info_2016 = pd.read_pickle('data2/horse_info_2016.pickle')
horse_info_2017 = pd.read_pickle('data2/horse_info_2017.pickle')
horse_info_2018 = pd.read_pickle('data2/horse_info_2018.pickle')
horse_info_2019 = pd.read_pickle('data2/horse_info_2019.pickle')
horse_info_2020 = pd.read_pickle('data2/horse_info_2020.pickle')
horse_info_2021 = pd.read_pickle('data2/horse_info_2021.pickle')
horse_info_2022 = pd.read_pickle('data2/horse_info_2022.pickle')
horse_info_2023 = pd.read_pickle('data2/horse_info_2023.pickle')
horse_info_2024 = pd.read_pickle('data2/horse_info_2024.pickle')
horse_info = pd.concat([
            horse_info_2006,\
            horse_info_2007,\
            horse_info_2008,\
            horse_info_2009,\
            horse_info_2010,\
            horse_info_2011,\
            horse_info_2012,\
            horse_info_2013,\
            horse_info_2014,\
            horse_info_2015,\
            horse_info_2016,\
            horse_info_2017,\
            horse_info_2018,\
            horse_info_2019,\
            horse_info_2020,\
            horse_info_2021,\
            horse_info_2022,\
            horse_info_2023,\
            horse_info_2024\
            ])

horse_peds_2000_2009 = pd.read_pickle('data2/horse_peds_2000-2009.pickle')
horse_peds_2010_2019 = pd.read_pickle('data2/horse_peds_2010-2019.pickle')
horse_peds_2020_2029 = pd.read_pickle('data2/horse_peds_2020-2029.pickle')
horse_peds = pd.concat([
            horse_peds_2000_2009,\
            horse_peds_2010_2019,\
            horse_peds_2020_2029\
            ])

race_return_2010_2014 = pd.read_pickle('data2/race_return_2010-2014.pickle')
race_return_2015_2019 = pd.read_pickle('data2/race_return_2015-2019.pickle')
race_return_2020_2024 = pd.read_pickle('data2/race_return_2020-2024.pickle')
race_return = pd.concat([
            race_return_2010_2014,\
            race_return_2015_2019,\
            race_return_2020_2024\
            ])

In [ ]:
#基本不要
horse_peds = pd.read_pickle('data2/horse_peds.pickle')

In [9]:
#基本不要
#race_info['data_sakusei_nengappi'].value_counts()
race_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36704 entries, 201845110105 to 202409020512
Data columns (total 62 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   record_id                  36704 non-null  object
 1   data_kubun                 36704 non-null  object
 2   data_sakusei_nengappi      36704 non-null  object
 3   kaisai_nen                 36704 non-null  object
 4   kaisai_tsukihi             36704 non-null  object
 5   keibajo_code               36704 non-null  object
 6   kaisai_kai                 36704 non-null  object
 7   kaisai_nichime             36704 non-null  object
 8   race_bango                 36704 non-null  object
 9   yobi_code                  36704 non-null  object
 10  tokubetsu_kyoso_bango      36704 non-null  object
 11  kyosomei_hondai            36704 non-null  object
 12  kyosomei_fukudai           36704 non-null  object
 13  kyosomei_kakkonai          36704 non-null  objec

# 1.Ｒａｃｅ＿Ｉｎｆｏ関連

In [ ]:
# 001-001
#　ｒａｃｅ＿ｉｎｆｏから必要な情報抽出　開催年は　テスト終了したら除外
race_info_sel = race_info[(race_info['kaisai_nen'] >= '2022') & (race_info['data_kubun'] == '7')]

In [ ]:
# 001-002
#　ｒａｃｅ＿ｉｎｆｏから必要な項目セレクト
race_info_sel = race_info_sel[[\
#                           'record_id',\
#                           'data_kubun',\
#                           'data_sakusei_nengappi',\
#                           'kaisai_nen',\
#                           'kaisai_tsukihi',\
#                           'keibajo_code',\
#                           'kaisai_kai',\
#                           'kaisai_nichime',\
#                           'race_bango',\
                           'grade_code',\
                           'kyoso_shubetsu_code',\
                           'kyoso_kigo_code',\
                           'juryo_shubetsu_code',\
#                           'kyoso_joken_code_2sai',\
#                           'kyoso_joken_code_3sai',\
#                           'kyoso_joken_code_4sai',\
#                           'kyoso_joken_code_5sai_ijo',\
                           'kyoso_joken_code',\
                           'kyori',\
                           'track_code',\
                           'shusso_tosu',\
                           'tenko_code',\
                           'babajotai_code_shiba',\
                           'babajotai_code_dirt',\
#                           'lap_time',\
#                           'zenhan_3f',\
#                           'zenhan_4f',\
#                           'kohan_3f',\
#                           'kohan_4f'\
                            ]]

In [ ]:
#001-003
#　ｒａｃｅ＿ｉｎｆｏにコード設定
race_info_sel['around'] = race_info_sel['track_code']
race_info_sel['race_type'] = race_info_sel['track_code']
race_info_sel['race_class'] = race_info_sel['kyoso_joken_code']
race_info_sel.replace({'around': ar_code_list},inplace=True)
race_info_sel.replace({'race_type': gr_code_list},inplace=True)
race_info_sel.replace({'race_class': class_code_list},inplace=True)
race_info_sel.replace({'grade_code': grade_code_list},inplace=True)
race_info_sel['race_class'] = race_info_sel.apply(\
        lambda x: x['race_class'] if x['grade_code'] == 'その他' else x['grade_code'] ,\
                    axis=1)
race_info_sel['baba_jotai'] = race_info_sel.apply(\
        lambda x: x['babajotai_code_shiba'] if x['race_type'] == '芝' else x['babajotai_code_dirt'] ,\
                    axis=1)
race_info_sel.replace({'baba_jotai': babajotai_code_list},inplace=True)
race_info_sel.replace({'tenko_code': tenko_code_list},inplace=True)
race_info_sel.replace({'juryo_shubetsu_code': juryo_shubetsu_code_list},inplace=True)
race_info_sel.replace({'kyoso_shubetsu_code': kyoso_shubetsu_code_list},inplace=True)

In [ ]:
#001-004
#　ｒａｃｅ＿ｉｎｆｏ　加工後、不要な項目削除
race_info_sel = race_info_sel.drop([\
                        'track_code',\
                        'kyoso_joken_code',\
                        'babajotai_code_shiba',\
                        'babajotai_code_dirt'\
                        ], axis=1)

In [ ]:
#001-005
#　新馬戦、障害戦は除外
race_info_sel = race_info_sel[(race_info_sel['race_class'] != '新馬')]
race_info_sel = race_info_sel[(race_info_sel['race_type'] != '障')]

In [ ]:
#001-006
###　初回必須　　0012-10
###　1回流した後処理するとＡＢＥＮＤ　　0012-10
###　----------horse_infoのインデックスを血統登録番号へ　　将来的には_１へ移す？
horse_info = horse_info.reset_index('race_id')
horse_info = horse_info.set_index('ketto_toroku_bango')


# 2.ｔａｉｓｈｏ＿ｒａｃｅ（レース結果関連）

In [ ]:
#002-001
###　初回　必須　　0011-10
###　------------------------------
### 対象のrace_resultsを抽出　－－＞　taisho_race　とする
###　------------------------------
###　----------分析用データの条件設定(ｔａｉｓｈｏ＿ｒａｃｅの処理)
taisho_race = race_results[(race_results['kaisai_nen'] >= '2022') & (race_results['data_kubun'] == '7')]
#taisho_race = race_results[race_results['kaisai_nen'] == '2023']
taisho_race['ketto_toroku_bango'] = taisho_race['ketto_toroku_bango'].astype('int')
#＞＞＞## 対象のrace_resultsのキー情報抽出
#＞＞＞taisho_race = taisho_race.loc[:, ["ketto_toroku_bango", "race_nengappi"]]
###　レース年月日を日付タイプへ　かつ変数名を　date　へ
taisho_race["date"] = pd.to_datetime(taisho_race["race_nengappi"])
taisho_race.drop(["race_nengappi"],axis=1,inplace=True)
taisho_race = taisho_race.reset_index('race_id')

taisho_race['zogen_sa'] = pd.to_numeric(taisho_race['zogen_sa'] ,errors='coerce')
taisho_race.fillna(0,inplace=True)
taisho_race['batai_zogen'] = taisho_race.apply(\
        lambda x: x['zogen_sa'] if x['zogen_fugo'] != '-' else x['zogen_sa'] * -1,\
                    axis=1)


In [ ]:
taisho_race['体重変化'].value_counts()

In [ ]:
###　ＡＢＥＮＤするのでながさない　　0012-10
###　----------horse_infoのインデックスを血統登録番号へ　　将来的には_１へ移す？
#horse_peds = horse_peds.reset_index()
#horse_peds = horse_peds.set_index('ketto_toroku_bango')

# 3.血統関連　今は飛ばす

In [ ]:
#003-001
horse_peds['ketto_toroku_bango'] = horse_peds['ketto_toroku_bango'].astype('int')
horse_peds = horse_peds.drop([\
                        'data_kubun',\
                        'record_id'\
                        ], axis=1)

In [ ]:
#基本不要
horse_peds.head()

In [ ]:
#基本不要
horse_info.tail()

In [ ]:
#　003-002　今後
# これながすとのちのmerge関数でＡＢＥＮＤするのでパス
taisho_race = pd.merge(taisho_race,horse_peds,\
                             left_on='ketto_toroku_bango',right_on='ketto_toroku_bango',how='left')

In [ ]:
#　003-003　今後
#  003-002　実行しなくても実行要
taisho_race = taisho_race.reset_index()
taisho_race = taisho_race.set_index('race_id')


# 4.マージ

In [ ]:
#基本不要
#horse_peds['ketto_toroku_bango'].info()
taisho_race.info()

In [ ]:
#　004-001
merged_results = pd.merge(race_info_sel,taisho_race,\
                             left_index=True,right_index=True,how='left')

In [ ]:
#基本不要
merged_results.head()

In [ ]:
#基本不要
merged_results['race_nengappi'].value_counts().sort_values()

In [ ]:
#基本不要
merged_results.loc['202306050811']

In [ ]:
#基本不要test = results_9R.loc['202306050801']

# 5.過去情報付与

In [ ]:
#  005-001
hr = HorseResults(horse_info)
results_3R = hr.merge_all(merged_results,n_samples=3)
results_3R.set_index('race_id', inplace=True)
results_5R = hr.merge_all(results_3R,n_samples=5)
results_5R.set_index('race_id', inplace=True)
results_9R = hr.merge_all(results_5R,n_samples=9)
results_9R.set_index('race_id', inplace=True)
results_all = hr.merge_all(results_9R,n_samples='all')
results_all.set_index('race_id', inplace=True)



In [ ]:
hr = HorseResults2(horse_info)
results_3R_high = hr.merge_all2(merged_results,n_samples=3)
results_3R_high.set_index('race_id', inplace=True)
results_5R_high = hr.merge_all2(results_3R_high,n_samples=5)
results_5R_high.set_index('race_id', inplace=True)
results_9R_high = hr.merge_all2(results_5R_high,n_samples=9)
results_9R_high.set_index('race_id', inplace=True)
results_all_high = hr.merge_all2(results_9R_high,n_samples='all')
results_all_high.set_index('race_id', inplace=True)

In [ ]:
#基本不要
results_all_high.loc['202306050811']

In [ ]:
#  005-002
###　オブジェクトをいったん変数へ
#model_input = hr.merge_all(taisho_race)
model_input = results_all
###　必要に応じて保存　日付変更
model_input.to_pickle('data/model/model_input_20240407.pickle')

In [ ]:
#  005-010
###　再開時、ここからロード
model_input = pd.read_pickle('data/model/model_input_20240407.pickle')

In [ ]:
#基本不要
model_input['kakutei_chakujun'].value_counts()

# 6.モデルインプット用に整形

In [ ]:
#  006-001
### 型変換　基本ｉｎｔ
#model_input['record_id'] = model_input['record_id'].astype(int)
model_input['keibajo_code'] = model_input['keibajo_code'].astype('category')
model_input['kaisai_kai'] = model_input['kaisai_kai'].astype(int)
model_input['kaisai_nichime'] = model_input['kaisai_nichime'].astype(int)
model_input['race_bango'] = model_input['race_bango'].astype(int)
model_input['wakuban'] = model_input['wakuban'].astype(int)
model_input['umaban'] = model_input['umaban'].astype(int)
model_input['ketto_toroku_bango'] = model_input['ketto_toroku_bango'].astype('category')
#model_input['bamei'] = model_input['bamei'].astype(int)
model_input['umakigo_code'] = model_input['umakigo_code'].astype('category')
model_input['seibetsu_code'] = model_input['seibetsu_code'].astype('category')
model_input['hinshu_code'] = model_input['hinshu_code'].astype('category')
model_input['barei'] = model_input['barei'].astype(int)
model_input['tozai_shozoku_code'] = model_input['tozai_shozoku_code'].astype('category')
model_input['chokyoshi_code'] = model_input['chokyoshi_code'].astype('category')
model_input['banushi_code'] = model_input['banushi_code'].astype('category')
model_input['futan_juryo'] = model_input['futan_juryo'].astype(int)
model_input['kishu_code'] = model_input['kishu_code'].astype('category')
model_input['bataiju'] = model_input['bataiju'].astype(int)
#model_input['zogen_fugo'] = model_input['zogen_fugo'].astype(int)
#model_input['zogen_sa'] = model_input['zogen_sa'].astype(int)
model_input['kakutei_chakujun'] = model_input['kakutei_chakujun'].astype(int)
model_input['soha_time'] = model_input['soha_time'].astype(int)
#model_input['chakusa_code_1'] = model_input['chakusa_code_1'].astype(int)
model_input['corner_1'] = model_input['corner_1'].astype(int)
model_input['corner_2'] = model_input['corner_2'].astype(int)
model_input['corner_3'] = model_input['corner_3'].astype(int)
model_input['corner_4'] = model_input['corner_4'].astype(int)
model_input['tansho_odds'] = model_input['tansho_odds'].astype(int)
model_input['tansho_ninkijun'] = model_input['tansho_ninkijun'].astype(int)
model_input['kakutoku_honshokin'] = model_input['kakutoku_honshokin'].astype(int)
#model_input['kakutoku_fukashokin'] = model_input['kakutoku_fukashokin'].astype(int)
model_input['kohan_4f'] = model_input['kohan_4f'].astype(int)
model_input['kohan_3f'] = model_input['kohan_3f'].astype(int)
#model_input['time_sa'] = model_input['time_sa'].astype(int)
#model_input['mining_kubun'] = model_input['mining_kubun'].astype('category')
#model_input['yoso_soha_time'] = model_input['yoso_soha_time'].astype(int)
#model_input['yoso_gosa_plus'] = model_input['yoso_gosa_plus'].astype('category')
#model_input['yoso_gosa_minus'] = model_input['yoso_gosa_minus'].astype('category')
#model_input['yoso_juni'] = model_input['yoso_juni'].astype(int)
#model_input['kyakushitsu_hantei'] = model_input['kyakushitsu_hantei'].astype('category')


In [ ]:
#基本不要
model_input['rank'].value_counts()

In [ ]:
#  006-002
###　目的変数の設定　4着以下はすべて４
#clip_rank = lambda x: x if x<4 else 4
clip_rank = lambda x: 1 if x<4 else 0
model_input['rank'] = model_input['kakutei_chakujun'].map(clip_rank)

###　着順０は除外
model_input = model_input[model_input['kakutei_chakujun'] > 0]

In [ ]:
#model_input['index'].value_counts()
model_input.info()

In [ ]:
#  006-003
###　説明変数候補以外はドロップ
model_input.drop(['record_id',\
                  'index',\
                  'data_kubun',\
                  'data_sakusei_nengappi',\
                  'kaisai_nen',\
                  'kaisai_tsukihi',\
                  'kaisai_nichime',\
                  'race_bango',\
                  'bamei',\
                  'zogen_fugo',\
                  'zogen_sa',\
                  'soha_time',\
                  'chakusa_code_1',\
#                  'date',\
                  'corner_1',\
                  'corner_2',\
                  'corner_3',\
                  'corner_4',\
                  'ketto_toroku_bango',\
                  'umakigo_code',\
                  'hinshu_code',\
                  'kakutei_chakujun',\
                  'tansho_odds',\
                  'tansho_ninkijun',\
                  'kakutoku_honshokin',\
                  'kakutoku_fukashokin',\
                  'kohan_4f',\
                  'kohan_3f',\
                  'time_sa',\
                  'mining_kubun',\
                  'yoso_soha_time',\
                  'yoso_gosa_plus',\
                  'yoso_gosa_minus',\
                  'yoso_juni',\
                  'grade_code',\
                  'kyoso_shubetsu_code',\
                  'kyoso_kigo_code',\
                  'juryo_shubetsu_code',\
#                  'kyori',\
                  'shusso_tosu',\
                  'tenko_code',\
                  'kyakushitsu_hantei'\
                  ],axis=1,inplace=True)

In [ ]:
#006-004
## カテゴリ変数をＤＵＭＭＹ化
model_input_d = pd.get_dummies(model_input)

In [ ]:
#006-005
###　初期値を０埋め
model_input_d.fillna(0,inplace=True)

ここからしばらく不要

In [ ]:
pred_df = pd.DataFrame({'pred':y_pred,'actual':y_test})

In [ ]:
#　pred予測1着　実際は？？
pred_df[pred_df['pred']==1]['actual'].value_counts()

# 7.ここからモデルゾーン

In [ ]:
train, test = split_data(model_input_d, test_size=0.3)
X_train = train.drop(["date", "rank"], axis=1)
y_train = train["rank"]
X_test = test.drop(["date", "rank"], axis=1)
y_test = test["rank"]

## 7.1 ランダムフォレスト

In [ ]:
#ランダムフォレストによる予測
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train), clf.score(X_test, y_test))

In [ ]:
y_pred = clf.predict_proba(X_test)[:,1]

In [ ]:
y_pred

In [ ]:
#ROC曲線の表示
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

#jupyterlabを使う場合、この2行はいらない
from jupyterthemes import jtplot
jtplot.style(theme="monokai")

fpr, tpr, thresholds = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr, marker="o")
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
plt.grid()
plt.show()


In [ ]:
#AUCスコアの表示
roc_auc_score(y_test, y_pred)
y_pred_train = clf.predict_proba(X_train)[:, 1]
roc_auc_score(y_train, y_pred_train)


In [ ]:
roc_auc_score(y_test, y_pred)

In [ ]:
#パラメータの調整
params = {
    "min_samples_split": 500,
    "max_depth": None,
    "n_estimators": 60,
    "criterion": "entropy",
    "class_weight": "balanced",
    "random_state": 100,
}

rf = RandomForestClassifier(**params)
rf.fit(X_train, y_train)
y_pred_train = rf.predict_proba(X_train)[:, 1]
y_pred = rf.predict_proba(X_test)[:, 1]
print(roc_auc_score(y_train, y_pred_train))
print(roc_auc_score(y_test, y_pred))

In [ ]:
#変数の重要度の表示
importances = pd.DataFrame(
    {"features": X_train.columns, "importance": rf.feature_importances_}
)
importances.sort_values("importance", ascending=False)[:20]

## 7.2勾配ブースティング木

In [ ]:
#LightGBMによる予測モデル作成
import lightgbm as lgb

params = {
    "num_leaves": 4,
    "n_estimators": 80,
    #'min_data_in_leaf': 15,
    "class_weight": "balanced",
    "random_state": 100,
}
lgb_clf = lgb.LGBMClassifier(**params)
lgb_clf.fit(X_train.values, y_train.values)
y_pred_train = lgb_clf.predict_proba(X_train)[:, 1]
y_pred = lgb_clf.predict_proba(X_test)[:, 1]
print(roc_auc_score(y_train, y_pred_train))
print(roc_auc_score(y_test, y_pred))


In [ ]:
#変数の重要度の表示
importances = pd.DataFrame(
    {"features": X_train.columns, "importance": lgb_clf.feature_importances_}
)
importances.sort_values("importance", ascending=False)[:20]

# 8.予想適用

In [ ]:
race_info = pd.read_pickle('data/shutsuba/race_info.pickle')

In [30]:
race_info_sel = race_info[(race_info['data_kubun'] == '2')]

In [13]:
#  1001-001
#　ｒａｃｅ＿ｉｎｆｏから必要な情報抽出　開催年は　テスト終了したら除外
race_info_sel = race_info[race_info['data_kubun'] == '7']

In [31]:
#  1001-002
#　ｒａｃｅ＿ｉｎｆｏから必要な項目セレクト
race_info_sel = race_info_sel[[\
#                           'record_id',\
#                           'data_kubun',\
#                           'data_sakusei_nengappi',\
#                           'kaisai_nen',\
#                           'kaisai_tsukihi',\
#                           'keibajo_code',\
#                           'kaisai_kai',\
#                           'kaisai_nichime',\
#                           'race_bango',\
                           'grade_code',\
                           'kyoso_shubetsu_code',\
                           'kyoso_kigo_code',\
                           'juryo_shubetsu_code',\
#                           'kyoso_joken_code_2sai',\
#                           'kyoso_joken_code_3sai',\
#                           'kyoso_joken_code_4sai',\
#                           'kyoso_joken_code_5sai_ijo',\
                           'kyoso_joken_code',\
                           'kyori',\
                           'track_code',\
                           'shusso_tosu',\
                           'tenko_code',\
                           'babajotai_code_shiba',\
                           'babajotai_code_dirt',\
#                           'lap_time',\
#                           'zenhan_3f',\
#                           'zenhan_4f',\
#                           'kohan_3f',\
#                           'kohan_4f'\
                            ]]

In [32]:
#  1001-003
#　ｒａｃｅ＿ｉｎｆｏにコード設定
race_info_sel['around'] = race_info_sel['track_code']
race_info_sel['race_type'] = race_info_sel['track_code']
race_info_sel['race_class'] = race_info_sel['kyoso_joken_code']
race_info_sel.replace({'around': ar_code_list},inplace=True)
race_info_sel.replace({'race_type': gr_code_list},inplace=True)
race_info_sel.replace({'race_class': class_code_list},inplace=True)
race_info_sel.replace({'grade_code': grade_code_list},inplace=True)
race_info_sel['race_class'] = race_info_sel.apply(\
        lambda x: x['race_class'] if x['grade_code'] == 'その他' else x['grade_code'] ,\
                    axis=1)
race_info_sel['baba_jotai'] = race_info_sel.apply(\
        lambda x: x['babajotai_code_shiba'] if x['race_type'] == '芝' else x['babajotai_code_dirt'] ,\
                    axis=1)
race_info_sel.replace({'baba_jotai': babajotai_code_list},inplace=True)
race_info_sel.replace({'tenko_code': tenko_code_list},inplace=True)
race_info_sel.replace({'juryo_shubetsu_code': juryo_shubetsu_code_list},inplace=True)
race_info_sel.replace({'kyoso_shubetsu_code': kyoso_shubetsu_code_list},inplace=True)

In [33]:
#  1001-004
#　ｒａｃｅ＿ｉｎｆｏ　加工後、不要な項目削除
race_info_sel = race_info_sel.drop([\
                        'track_code',\
                        'kyoso_joken_code',\
                        'babajotai_code_shiba',\
                        'babajotai_code_dirt'\
                        ], axis=1)

In [ ]:
#  1001-005
#　不要

In [17]:
#  1001-006
###　初回必須　　0012-10
###　1回流した後処理するとＡＢＥＮＤ　　0012-10
###　----------horse_infoのインデックスを血統登録番号へ　　将来的には_１へ移す？
horse_info = horse_info.reset_index('race_id')
horse_info = horse_info.set_index('ketto_toroku_bango')


In [27]:
#  1002-001
###　初回　必須　　0011-10
###　------------------------------
### 対象のrace_resultsを抽出　－－＞　taisho_race　とする
###　------------------------------
###　----------分析用データの条件設定(ｔａｉｓｈｏ＿ｒａｃｅの処理)
taisho_race = race_results[(race_results['data_kubun'] == '2')]
#taisho_race = race_results[race_results['kaisai_nen'] == '2023']
taisho_race['ketto_toroku_bango'] = taisho_race['ketto_toroku_bango'].astype('int')
#＞＞＞## 対象のrace_resultsのキー情報抽出
#＞＞＞taisho_race = taisho_race.loc[:, ["ketto_toroku_bango", "race_nengappi"]]
###　レース年月日を日付タイプへ　かつ変数名を　date　へ
taisho_race["date"] = pd.to_datetime(taisho_race["race_nengappi"])
taisho_race.drop(["race_nengappi"],axis=1,inplace=True)
taisho_race = taisho_race.reset_index('race_id')


C:\Users\mk198\AppData\Local\Temp\ipykernel_25272\3302478093.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taisho_race['ketto_toroku_bango'] = taisho_race['ketto_toroku_bango'].astype('int')
C:\Users\mk198\AppData\Local\Temp\ipykernel_25272\3302478093.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taisho_race["date"] = pd.to_datetime(taisho_race["race_nengappi"])
C:\Users\mk198\AppData\Local\Temp\ipykernel_25272\3302478093.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [28]:
taisho_race = taisho_race.reset_index()
taisho_race = taisho_race.set_index('race_id')

taisho_race = taisho_race.loc['202409020611']
#taisho_race_sel.info()


In [29]:
taisho_race

,index,record_id,data_kubun,data_sakusei_nengappi,kaisai_nen,kaisai_tsukihi,keibajo_code,kaisai_kai,kaisai_nichime,race_bango,...,kohan_4f,kohan_3f,time_sa,mining_kubun,yoso_soha_time,yoso_gosa_plus,yoso_gosa_minus,yoso_juni,kyakushitsu_hantei,date
race_id,,,,,,,,,,,,,,,,,,,,,
202409020611,0,SE,2,20240405,2024,0407,09,02,06,11,...,000,000,0000,0,00000,0000,0000,00,0,2024-04-07
202409020611,1,SE,2,20240405,2024,0407,09,02,06,11,...,000,000,0000,0,00000,0000,0000,00,0,2024-04-07
202409020611,2,SE,2,20240405,2024,0407,09,02,06,11,...,000,000,0000,0,00000,0000,0000,00,0,2024-04-07
202409020611,3,SE,2,20240405,2024,0407,09,02,06,11,...,000,000,0000,0,00000,0000,0000,00,0,2024-04-07
202409020611,4,SE,2,20240405,2024,0407,09,02,06,11,...,000,000,0000,0,00000,0000,0000,00,0,2024-04-07
202409020611,5,SE,2,20240405,2024,0407,09,02,06,11,...,000,000,0000,0,00000,0000,0000,00,0,2024-04-07
202409020611,6,SE,2,20240405,2024,0407,09,02,06,11,...,000,000,0000,0,00000,0000,0000,00,0,2024-04-07
202409020611,7,SE,2,20240405,2024,0407,09,02,06,11,...,000,000,0000,0,00000,0000,0000,00,0,2024-04-07
202409020611,8,SE,2,20240405,2024,0407,09,02,06,11,...,000,000,0000,0,00000,0000,0000,00,0,2024-04-07


In [ ]:
#  1003-001
#　今後　現時点不要
horse_peds['ketto_toroku_bango'] = horse_peds['ketto_toroku_bango'].astype('int')
horse_peds = horse_peds.drop([\
                        'data_kubun',\
                        'record_id'\
                        ], axis=1)

In [ ]:
#　1003-002　今後
# これながすとのちのmerge関数でＡＢＥＮＤするのでパス
taisho_race = pd.merge(taisho_race,horse_peds,\
                             left_on='ketto_toroku_bango',right_on='ketto_toroku_bango',how='left')

In [ ]:
#　1003-003　今後
taisho_race = taisho_race.reset_index()
taisho_race = taisho_race.set_index('race_id')


In [34]:
#　1004-001
merged_results = pd.merge(race_info_sel,taisho_race,\
                             left_index=True,right_index=True,how='left')

In [26]:
horse_info.head()

,race_id,kakutei_chakujun,soha_time,corner_1,corner_4,tansho_odds,tansho_ninkijun,kakutoku_honshokin,kohan_3f,time_sa,yoso_juni,kyakushitsu_hantei,race_nengappi
ketto_toroku_bango,,,,,,,,,,,,,
0,2013G0000004,1,2272,0,0,0,0,0,0,0,0,0,20131208
0,2013G0000004,8,0,0,0,0,0,0,0,0,0,0,20131208
0,2015B6000007,1,3231,0,0,0,0,0,0,0,0,0,20151103
0,2015B6000007,16,0,0,0,0,0,0,0,0,0,0,20151103
0,2015B6000007,12,0,0,0,0,0,0,0,0,0,0,20151103


In [35]:
#  1005-001
hr = HorseResults(horse_info)
results_3R = hr.merge_all(merged_results,n_samples=3)
results_3R.set_index('race_id', inplace=True)
results_5R = hr.merge_all(results_3R,n_samples=5)
results_5R.set_index('race_id', inplace=True)
results_9R = hr.merge_all(results_5R,n_samples=9)
results_9R.set_index('race_id', inplace=True)
results_all = hr.merge_all(results_9R,n_samples='all')
results_all.set_index('race_id', inplace=True)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [36]:
results_all

,grade_code,kyoso_shubetsu_code,kyoso_kigo_code,juryo_shubetsu_code,kyori,shusso_tosu,tenko_code,around,race_type,race_class,...,kohan_3f_3R,chakujun_5R,shokin_5R,kohan_3f_5R,chakujun_9R,shokin_9R,kohan_3f_9R,chakujun_allR,shokin_allR,kohan_3f_allR
race_id,,,,,,,,,,,,,,,,,,,,,
202409020611,Ｇ１,サラ３歳,N21,馬齢,1600,00,未設定,右,芝,Ｇ１,...,346.333333,6.250000,68000.000000,346.000000,6.250000,68000.000000,346.000000,6.250000,68000.000000,346.000000
202409020611,Ｇ１,サラ３歳,N21,馬齢,1600,00,未設定,右,芝,Ｇ１,...,335.666667,1.333333,154666.666667,335.666667,1.333333,154666.666667,335.666667,1.333333,154666.666667,335.666667
202409020611,Ｇ１,サラ３歳,N21,馬齢,1600,00,未設定,右,芝,Ｇ１,...,344.333333,1.666667,151000.000000,344.333333,1.666667,151000.000000,344.333333,1.666667,151000.000000,344.333333
202409020611,Ｇ１,サラ３歳,N21,馬齢,1600,00,未設定,右,芝,Ｇ１,...,352.000000,3.800000,72400.000000,346.200000,4.166667,60333.333333,344.666667,4.166667,60333.333333,344.666667
202409020611,Ｇ１,サラ３歳,N21,馬齢,1600,00,未設定,右,芝,Ｇ１,...,350.000000,4.400000,70600.000000,349.200000,4.166667,61833.333333,349.500000,4.166667,61833.333333,349.500000
202409020611,Ｇ１,サラ３歳,N21,馬齢,1600,00,未設定,右,芝,Ｇ１,...,345.000000,3.666667,61666.666667,345.000000,3.666667,61666.666667,345.000000,3.666667,61666.666667,345.000000
202409020611,Ｇ１,サラ３歳,N21,馬齢,1600,00,未設定,右,芝,Ｇ１,...,342.333333,2.600000,139600.000000,342.600000,2.666667,119333.333333,344.000000,2.666667,119333.333333,344.000000
202409020611,Ｇ１,サラ３歳,N21,馬齢,1600,00,未設定,右,芝,Ｇ１,...,340.666667,1.600000,193000.000000,344.200000,1.833333,163833.333333,342.333333,1.833333,163833.333333,342.333333
202409020611,Ｇ１,サラ３歳,N21,馬齢,1600,00,未設定,右,芝,Ｇ１,...,334.333333,1.000000,344000.000000,334.333333,1.000000,344000.000000,334.333333,1.000000,344000.000000,334.333333


In [ ]:
#  1005-002
###　オブジェクトをいったん変数へ
#model_input = hr.merge_all(taisho_race)
model_input = results_all
###　必要に応じて保存　日付変更
model_input.to_pickle('data/model/shutsuba_20240407.pickle')

In [ ]:
#  1006-001
### 型変換　基本ｉｎｔ
#model_input['record_id'] = model_input['record_id'].astype(int)
model_input['keibajo_code'] = model_input['keibajo_code'].astype('category')
model_input['kaisai_kai'] = model_input['kaisai_kai'].astype(int)
model_input['kaisai_nichime'] = model_input['kaisai_nichime'].astype(int)
model_input['race_bango'] = model_input['race_bango'].astype(int)
model_input['wakuban'] = model_input['wakuban'].astype(int)
model_input['umaban'] = model_input['umaban'].astype(int)
model_input['ketto_toroku_bango'] = model_input['ketto_toroku_bango'].astype('category')
#model_input['bamei'] = model_input['bamei'].astype(int)
model_input['umakigo_code'] = model_input['umakigo_code'].astype('category')
model_input['seibetsu_code'] = model_input['seibetsu_code'].astype('category')
model_input['hinshu_code'] = model_input['hinshu_code'].astype('category')
model_input['barei'] = model_input['barei'].astype(int)
model_input['tozai_shozoku_code'] = model_input['tozai_shozoku_code'].astype('category')
model_input['chokyoshi_code'] = model_input['chokyoshi_code'].astype('category')
model_input['banushi_code'] = model_input['banushi_code'].astype('category')
model_input['futan_juryo'] = model_input['futan_juryo'].astype(int)
model_input['kishu_code'] = model_input['kishu_code'].astype('category')
model_input['bataiju'] = model_input['bataiju'].astype(int)
#model_input['zogen_fugo'] = model_input['zogen_fugo'].astype(int)
#model_input['zogen_sa'] = model_input['zogen_sa'].astype(int)
model_input['kakutei_chakujun'] = model_input['kakutei_chakujun'].astype(int)
model_input['soha_time'] = model_input['soha_time'].astype(int)
#model_input['chakusa_code_1'] = model_input['chakusa_code_1'].astype(int)
model_input['corner_1'] = model_input['corner_1'].astype(int)
model_input['corner_2'] = model_input['corner_2'].astype(int)
model_input['corner_3'] = model_input['corner_3'].astype(int)
model_input['corner_4'] = model_input['corner_4'].astype(int)
model_input['tansho_odds'] = model_input['tansho_odds'].astype(int)
model_input['tansho_ninkijun'] = model_input['tansho_ninkijun'].astype(int)
model_input['kakutoku_honshokin'] = model_input['kakutoku_honshokin'].astype(int)
#model_input['kakutoku_fukashokin'] = model_input['kakutoku_fukashokin'].astype(int)
model_input['kohan_4f'] = model_input['kohan_4f'].astype(int)
model_input['kohan_3f'] = model_input['kohan_3f'].astype(int)
#model_input['time_sa'] = model_input['time_sa'].astype(int)
#model_input['mining_kubun'] = model_input['mining_kubun'].astype('category')
#model_input['yoso_soha_time'] = model_input['yoso_soha_time'].astype(int)
#model_input['yoso_gosa_plus'] = model_input['yoso_gosa_plus'].astype('category')
#model_input['yoso_gosa_minus'] = model_input['yoso_gosa_minus'].astype('category')
#model_input['yoso_juni'] = model_input['yoso_juni'].astype(int)
#model_input['kyakushitsu_hantei'] = model_input['kyakushitsu_hantei'].astype('category')


In [ ]:
# 1006-002 不要

In [ ]:
#  1006-003
###　説明変数候補以外はドロップ
model_input.drop(['record_id',\
                  'data_kubun',\
                  'data_sakusei_nengappi',\
                  'kaisai_nen',\
                  'kaisai_tsukihi',\
                  'kaisai_nichime',\
                  'race_bango',\
                  'bamei',\
                  'zogen_fugo',\
                  'zogen_sa',\
                  'soha_time',\
                  'chakusa_code_1',\
#                  'date',\
                  'corner_1',\
                  'corner_2',\
                  'corner_3',\
                  'corner_4',\
                  'ketto_toroku_bango',\
                  'umakigo_code',\
                  'hinshu_code',\
                  'kakutei_chakujun',\
                  'tansho_odds',\
                  'tansho_ninkijun',\
                  'kakutoku_honshokin',\
                  'kakutoku_fukashokin',\
                  'kohan_4f',\
                  'kohan_3f',\
                  'time_sa',\
                  'mining_kubun',\
                  'yoso_soha_time',\
                  'yoso_gosa_plus',\
                  'yoso_gosa_minus',\
                  'yoso_juni',\
                  'grade_code',\
                  'kyoso_shubetsu_code',\
                  'kyoso_kigo_code',\
                  'juryo_shubetsu_code',\
                  'kyori',\
                  'shusso_tosu',\
                  'tenko_code',\
                  'kyakushitsu_hantei'\
                  ],axis=1,inplace=True)

In [ ]:
#  1006-004
## カテゴリ変数をＤＵＭＭＹ化
model_input_d = pd.get_dummies(model_input)

In [ ]:
#  1006-005
###　初期値を０埋め
model_input_d.fillna(0,inplace=True)

In [ ]:
###　モデル実施　データをトレーニング、テストにわける
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = model_input_d.drop(['rank'],axis=1)
y = model_input_d['rank']

X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size=0.3,random_state=0)

In [ ]:
model = LogisticRegression()
model.fit(X_train_rus,y_train_rus)

print(model.score(X_train,y_train),model.score(X_test,y_test))

In [ ]:
#y_pred = model.predict(X_test)
y_pred = model.predict_proba(X_test)

In [ ]:
y_pred

In [ ]:
pred_df = pd.DataFrame({'pred':y_pred,'actual':y_test})

In [ ]:
pred_df[pred_df['pred']==1]['actual'].value_counts()

In [ ]:
coefs = pd.Series(model.coef_[0],index=X.columns).sort_values()

In [ ]:
coefs

In [ ]:
#model_input[model_input['race_id'] == '2023010506010101'].to_csv('model_input_sel.csv')
model_input.loc['2023010506010101'].to_csv('model_input_sel.csv')


In [ ]:
###　？？？
sample_date = taisho_race['date'][0]

##
## date taisho_race の　date　を　日付で　ソート　＆　重複けし
## 日付がなくなるまで　ＦＯＲで繰り返す
sample_date

In [ ]:
horse_id_list = taisho_race[taisho_race['date'] == sample_date]['ketto_toroku_bango'].unique()

In [ ]:
#日付取得
sample_date = taisho_race['date'][0]
#対象馬リスト　その日走った馬
horse_id_list = taisho_race[taisho_race['date'] == sample_date]['ketto_toroku_bango'].unique()
#　LOCがうまくいかないのでＬＩＳＴ２作成
horse_id_list2 = pd.DataFrame(horse_id_list,columns=['ketto_toroku_bango'])
#type(horse_id_list2)
horse_id_list2['ketto_toroku_bango'] = horse_id_list2['ketto_toroku_bango'].astype('int')

#　候補ｒｅｓｕｌｔｓ　を自力で作成
kouho_results = pd.merge(horse_id_list2,hr.horse_results,left_on='ketto_toroku_bango',right_on='ketto_toroku_bango',how='left')
#hr.horse_results[(hr.horse_results.index == '2016106582')]
kouho_results = kouho_results.set_index('ketto_toroku_bango')

In [ ]:
#hr.horse_results.loc[horse_id_list2]
hr.preprocessing()

In [ ]:
target_df = kouho_results
target_df[target_df['date'] < sample_date].groupby(level=0)[['kakutei_chakujun','kakutoku_honshokin','kohan_3f']].mean()

In [ ]:
#horse_info['race_nengappi'].value_counts()
#horse_info['date'] = pd.to_datetime(horse_info["race_nengappi"])
#horse_info.isnull().sum()
horse_info['race_nengappi']
taisho_race

In [ ]:
horse_info = pd.read_pickle('data/shutsuba/horse_info.pickle')
#　馬情報テーブルにレース年月日追加
horse_info = horse_info[horse_info['data_kubun'] == '7']
#　馬情報項目セレクト
race_data_sel = horse_info[['ketto_toroku_bango',\
                           'kakutei_chakujun',\
                           'corner_1',\
                           'corner_4',\
                           'tansho_odds',\
                            'tansho_ninkijun',\
                           'kakutoku_honshokin',\
                            'kohan_3f',\
                            'time_sa',\
                            'yoso_juni',\
                            'kyakushitsu_hantei',\
                            'race_nengappi']]
race_data_sel['ketto_toroku_bango'] = race_data_sel['ketto_toroku_bango'].astype('int')
race_data_sel['kakutei_chakujun'] = race_data_sel['kakutei_chakujun'].astype('int')
race_data_sel['corner_1'] = race_data_sel['corner_1'].astype('int')
race_data_sel['corner_4'] = race_data_sel['corner_4'].astype('int')
race_data_sel['tansho_odds'] = race_data_sel['tansho_odds'].astype('int')
race_data_sel['tansho_ninkijun'] = race_data_sel['tansho_ninkijun'].astype('int')
race_data_sel['kakutoku_honshokin'] = race_data_sel['kakutoku_honshokin'].astype('int')
race_data_sel['time_sa'] = race_data_sel['time_sa'].astype('int')
race_data_sel['kohan_3f'] = race_data_sel['kohan_3f'].astype('int')
race_data_sel['yoso_juni'] = race_data_sel['yoso_juni'].astype('int')
race_data_sel['kyakushitsu_hantei'] = race_data_sel['kyakushitsu_hantei'].astype('int')
race_data_sel['race_nengappi'] = race_data_sel['race_nengappi'].astype('int')
race_data_sel.isnull().sum()
horse_info['tansho_odds'].value_counts

In [ ]:
race_data_sel['tansho_odds'].value_counts()


In [ ]:
horse_info = pd.read_csv('horse_info.csv')
horse_info['tansho_odds'].value_counts()

In [ ]:
### key_info 
taisho_race = race_data[race_data['kaisai_nen'] == '2023']
taisho_race = taisho_race.loc[:, ["ketto_toroku_bango", "race_nengappi"]]
taisho_race.head